In [ ]:
import asyncio
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

import pandas as pd

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.services.larp_client import LarpClient
from geckoterminal_py import GeckoTerminalAsyncClient
import pandas as pd
import asyncio


client = LarpClient()
gt = GeckoTerminalAsyncClient()

In [ ]:
# Criteria
MIN_POOL_AGE_DAYS = 7
MAX_POOL_AGE_DAYS = 720
MIN_MARKET_CAP = 1_000_000
MAX_MARKET_CAP = 100_000_000
MIN_VOLUME_24H = 100_000
MIN_LIQUIDITY = 10_000
NETWORK = "solana"
DEX = "meteora"
QUOTE_ASSET = "SOL"

In [ ]:
pools = await gt.get_top_pools_by_network_dex(NETWORK, DEX)

In [ ]:
pools["market_cap_usd"] = pd.to_numeric(pools["market_cap_usd"])
pools["volume_usd_h24"] = pd.to_numeric(pools["volume_usd_h24"])
pools["reserve_in_usd"] = pd.to_numeric(pools["reserve_in_usd"])
pools["pool_created_at"] = pd.to_datetime(pools["pool_created_at"]).dt.tz_localize(None)
pools["base"] = pools["name"].apply(lambda x: x.split("/")[0].strip())
pools["quote"] = pools["name"].apply(lambda x: x.split("/")[1].strip())
pools["volume_liquidity_ratio"] = pools["volume_usd_h24"] / pools["reserve_in_usd"]

In [ ]:
pools

In [ ]:
from datetime import datetime, timedelta

min_pool_date_created = datetime.utcnow() - timedelta(days=MIN_POOL_AGE_DAYS)
max_pool_date_created = datetime.utcnow() - timedelta(days=MAX_POOL_AGE_DAYS)
pools_filtered = pools[(pools["market_cap_usd"] > MIN_MARKET_CAP) & (pools["market_cap_usd"] < MAX_MARKET_CAP) & (pools["volume_usd_h24"] > MIN_VOLUME_24H) & (pools["reserve_in_usd"] > MIN_LIQUIDITY) & (pools["pool_created_at"] < min_pool_date_created) & (pools["pool_created_at"] > max_pool_date_created) & (pools["quote"] == QUOTE_ASSET)]
fee_rates = []
tick_spacings = []
liquidity = []
for i, pool in pools_filtered.iterrows():
    try:
        pool_info = await client.get_pool_info(pool["address"])
        fee_rates.append(pool_info["feeRateBps"])
        tick_spacings.append(pool_info["tickSpacing"])
        liquidity.append(pool_info["liquidity"])
        await asyncio.sleep(2)
    except:
        fee_rates.append(None)
        tick_spacings.append(None)
        liquidity.append(None)
        print(f"Error with pool {pool['address']}")
pools_filtered["fee_rate"] = fee_rates
pools_filtered["tick_spacing"] = tick_spacings
pools_filtered["liquidity"] = liquidity

In [ ]:
pools_filtered.sort_values("volume_liquidity_ratio", ascending=False, inplace=True)
pools_filtered

In [ ]:
from core.data_structures.candles import Candles


async def get_candles(pool, interval):
    trading_pair = f"{pool['base']}/{pool['quote']}"
    address = pool["address"]
    ohlc = await gt.get_ohlcv(NETWORK, address, interval, currency="token")
    ohlc.index = pd.to_datetime(ohlc["timestamp"], unit="s")
    return Candles(candles_df=ohlc, connector_name="orca", trading_pair=trading_pair, interval=interval)

def add_position_metrics(candle, lookback_periods):
    df = candle.data
    results = []
    
    for period in lookback_periods:
        df_period = df.iloc[-period:]
        max_price = df_period['high'].max()
        min_price = df_period['low'].min()
        range_price = max_price - min_price
        range_price_pct = (max_price - df_period['close'].iloc[-1]) / df_period['close'].iloc[-1]
        current_position = (max_price - df_period['close'].iloc[-1]) / range_price if range_price != 0 else 0
        
        results.append({
            'period': period,
            'max_price': max_price,
            'min_price': min_price,
            'current_position': current_position,
            'range_price_pct': range_price_pct,
            'current_price': df_period['close'].iloc[-1]
        })
    
    return results

In [ ]:
pools_data = {}
interval = "1d"

for i, pool in pools_filtered.iterrows():
    candle = await get_candles(pool, interval)
    pools_data[pool["address"]] = {
        "candle": candle,
        "metrics": add_position_metrics(candle, [len(candle.data)])[0],
        "pool": pool
    }

In [ ]:
# SORT BY CURRENT POSITION
pools_data = dict(sorted(pools_data.items(), key=lambda item: item[1]['metrics']['current_position'], reverse=True))


In [ ]:
potential_positions = []

for pool_id, pool_data in pools_data.items():
    candle = pool_data["candle"]
    metrics = pool_data["metrics"]
    fig = candle.fig()
    period = metrics['period']
    max_price = metrics['max_price']
    min_price = metrics['min_price']
    current_position = metrics['current_position']
    current_price = metrics['current_price']
    # tick_spacing = pool_data["pool"]["tick_spacing"]
    # if current_position < 0.8 or pd.isna(tick_spacing):
    if current_position < 0.8:
        continue
    potential_positions.append({
        "base_symbol": pool_data["pool"]["base"],
        "quote_symbol": pool_data["pool"]["quote"],
        # "tick_spacing": int(tick_spacing),
        "lower_price": str(min_price),
        "upper_price": str(current_price),
        "number_of_positions": 1,
    })
    # Add horizontal lines for max price, min price, and current position
    fig.add_hline(y=max_price, line_dash="dash", line_color="green", annotation_text=f"Max Price ({period})", annotation_position="right")
    fig.add_hline(y=min_price, line_dash="dash", line_color="red", annotation_text=f"Min Price ({period})", annotation_position="right")
    fig.add_hline(y=current_price, line_dash="dash", line_color="orange", annotation_text=f"Current Position ({current_position})", annotation_position="right")

    fig.update_layout(title=f"{candle.trading_pair} - Price Range and Current Position",
                      xaxis_title="Date",
                      yaxis_title="Price",
                      showlegend=True)

    fig.show()


In [ ]:
potential_positions

In [ ]:
position_bundle = await client.create_orca_position_bundle()

In [ ]:
position_bundle

In [ ]:
pools_filtered

In [ ]:
position_to_create = potential_positions[1]
position_to_create["position_bundle_address"] = position_bundle["positionBundleAddress"]
position_to_create

In [ ]:
await client.open_orca_positions_in_bundle(
    **position_to_create
)